In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow.keras as keras

from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image           # pip install pillow
from collections import defaultdict
import time
import cv2
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
# import tensorflow.keras as keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout,Average
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image
from collections import defaultdict
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import os
from sklearn.metrics import accuracy_score
# import tensorflow.keras as keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD

In [2]:

TARGET_SIZE=(224,224)     
INPUT_SIZE=(224,224,3)    
BATCHSIZE=32          

test_datagen = ImageDataGenerator(rescale=1./255)

        
validation_generator = test_datagen.flow_from_directory(
            './test_224/',
            target_size=TARGET_SIZE,
            batch_size=BATCHSIZE,
            class_mode='categorical')

Found 261 images belonging to 5 classes.


In [3]:
#generate labels indicating class
label=validation_generator.class_indices
label={v: k for k, v in label.items()}

In [4]:
 #load model 1: xception
xception_model = load_model('./xception.h5')
#load model 2: VGG16
vgg_model = load_model('./VGG16.h5')
#load model 3: VGG19
vgg19_model = load_model('./VGG19.h5')
#load model 4: inception
incep_model = load_model('./inception.h5')
#load model 5: inceptionresnet
inres_model = load_model('./inceptionresnet.h5')

In [6]:
model1=Model(inputs=[xception_model.layers[0].get_input_at(0)],outputs=xception_model.get_layer('dense_1').output,name='xception')
model2=Model(inputs=[vgg_model.layers[0].get_input_at(0)],outputs=vgg_model.get_layer('dense_3').output,name='vgg')
model3=Model(inputs=[vgg19_model.layers[0].get_input_at(0)],outputs=vgg19_model.get_layer('dense_1').output,name='vgg19')
model4=Model(inputs=[incep_model.layers[0].get_input_at(0)],outputs=incep_model.get_layer('dense_3').output,name='incep')
model5=Model(inputs=[inres_model.layers[0].get_input_at(0)],outputs=inres_model.get_layer('dense_5').output,name='inres')

In [7]:
ensemble_model=load_model('./ensemble.h5')

In [8]:
#read images from validation folder


rootdir = './test_224_shuffled/'

test_laels = []
test_images=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
            continue
        test_laels.append(subdir.split('/')[-1])
        test_images.append(os.path.join(subdir, file))
        
print(test_laels[0],test_images[0])

0 ./test_224_shuffled/0\10064.png


In [9]:
#test the averaging model on the validation set

predict=[]
length=len(test_images)
t1 = time.time()
for i in range((length//127)+1):
    inputimg=test_images[127*i:127*(i+1)]
    test_batch=[]
    for path in inputimg:
        thisimg=np.array(Image.open(path))/255
        test_batch.append(thisimg)
    #print(i, np.array(test_batch).shape)
    ensemble_model_batch=ensemble_model.predict(np.array(test_batch))
    predict_batch=list(np.argmax(ensemble_model_batch,axis=1))
    predict_batch=[label[con] for con in predict_batch]
    predict.append(predict_batch)

predict=sum(predict,[])

t2 = time.time()
print('The testing time is :%f seconds' % (t2-t1))

The testing time is :62.931097 seconds


In [10]:

acc=accuracy_score(test_laels,predict)
print('Concatenation accuracy:%s'%acc)

Concatenation accuracy:0.25654450261780104


In [11]:
acc=accuracy_score(test_laels,predict)
pre=precision_score(test_laels,predict,average='weighted')
re=recall_score(test_laels,predict,average='weighted')
f1=f1_score(test_laels,predict,average='weighted')
print('ensemble accuracy: %s'%acc)
print('precision: %s'%pre)
print('recall: %s'%re)
print('f1: %s'%f1)

print(confusion_matrix(test_laels, predict))
target_names = ['0', '1','2','3','4']
print(classification_report(test_laels, predict, target_names=target_names))

ensemble accuracy: 0.25654450261780104
precision: 0.22825098280460718
recall: 0.25654450261780104
f1: 0.21097660272925542
[[39  0  6 98  6]
 [26  0  5 81 10]
 [16  0  6 54 14]
 [21  0  8 74  6]
 [22  0  9 44 28]]
              precision    recall  f1-score   support

           0       0.31      0.26      0.29       149
           1       0.00      0.00      0.00       122
           2       0.18      0.07      0.10        90
           3       0.21      0.68      0.32       109
           4       0.44      0.27      0.34       103

    accuracy                           0.26       573
   macro avg       0.23      0.26      0.21       573
weighted avg       0.23      0.26      0.21       573



In [12]:
label_map = {0: 'Benign', 1: 'DDoS', 2: 'DoS Hulk', 3: 'FTP-Patator', 4: 'SSH-Patator'}

In [13]:
y_true_list = [int(i) for i in test_laels]
y_pred_list = [int(i) for i in predict]

In [14]:

y_true_mapped = [label_map[y] for y in y_true_list]
y_pred_mapped = [label_map[y] for y in y_pred_list]

report = classification_report(y_true_mapped, y_pred_mapped, zero_division=1, output_dict=True)

cm = confusion_matrix(y_true_mapped, y_pred_mapped)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

for label, metrics in report.items():
    if isinstance(metrics, dict):  # skip accuracy, macro avg, etc.
        print(f"\nClass: {label}")
        for metric_name, value in metrics.items():
            print(f"  {metric_name}: {value:.3f}")



for class_label, acc in zip(report.keys(), per_class_accuracy):
    if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"Class {class_label} Accuracy: {acc * 100:.3f}%")
        
        
        


Class: Benign
  precision: 0.315
  recall: 0.262
  f1-score: 0.286
  support: 149.000

Class: DDoS
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 122.000

Class: DoS Hulk
  precision: 0.176
  recall: 0.067
  f1-score: 0.097
  support: 90.000

Class: FTP-Patator
  precision: 0.211
  recall: 0.679
  f1-score: 0.322
  support: 109.000

Class: SSH-Patator
  precision: 0.438
  recall: 0.272
  f1-score: 0.335
  support: 103.000

Class: macro avg
  precision: 0.428
  recall: 0.256
  f1-score: 0.208
  support: 573.000

Class: weighted avg
  precision: 0.441
  recall: 0.257
  f1-score: 0.211
  support: 573.000
Class Benign Accuracy: 26.174%
Class DDoS Accuracy: 0.000%
Class DoS Hulk Accuracy: 6.667%
Class FTP-Patator Accuracy: 67.890%
Class SSH-Patator Accuracy: 27.184%
